### Live Cell Imaging File Copy

This notebook intiates processing  image files by first copying them to the exacloud gscratch storage.

In [1]:
#setup libraries
import numpy as np
import pandas as pd
import os, re, glob, shutil, sys

In [9]:
directory_name = "AU565_drug_screen"
directory_name = "HCC1143_drug_screen"
directory_name = "21MT1_drug_screen"
#directory_name = "AU_I_L_006_01_1"

#directory_name = "AU565_Drug_Screen_Project" #used for files stored on graylab_share
plate_name = "AU00601"
plate_name = "HC01401"
plate_name = "2100701"

def get_file_df(plate_name):
    print("getting files for "+plate_name)
    #data_path =  '/home/groups/graylab_share/OMERO.rdsStore/liby/'+directory_name+'/'+plate_name+"/"+plate_name+'_IMAGES'
    data_path =  '/home/groups/heiserlab_genomics/home/liby/'+directory_name+'/'+plate_name+"/"+plate_name+'_IMAGES'
    #data_path =  '/home/groups/heiserlab_genomics/home/liby/'+directory_name+'/'+plate_name+'_IMAGES'
    #data_path = "/home/groups/heiserlab_genomics/home/grossse/AU565 Project/AU565 Image Files/AU_I_L_006_01_1/AU_I_L_006_01_1_IMAGES"
    data_paths = glob.glob(os.path.join(data_path,"*.tif"),recursive=False)
    df = pd.DataFrame(data_paths, columns =['path'])
    df['filename'] = df['path'].str.extract('(\w*.tif)')
    df['plateID'] = df['filename'].str.extract('(^[a-zA-Z0-9]*)')
    df['channel'] = df['filename'].str.extract('(_[PRG]_)')
    df['channel'] = df['channel'].str.extract('([PRG])')
    df['well'] = df['filename'].str.extract('(_[A-Z][0-9]+_)')
    df['well'] = df['well'].str.extract('([A-Z][0-9]+)')
    df['field'] = df['filename'].str.extract('(_[0-9]+_)')
    df['field'] = df['field'].str.extract('([0-9]+)')
    df['time'] = df['filename'].str.extract('(_[a-z0-9]*.tif)')
    df['time'] = df['time'].str.replace('[_(.tif)]','',regex = True)
    return df

def make_directories(df, dest_path):
    print("making directories in "+dest_path)
    df_directories = df[['plateID', 'well', 'field']].drop_duplicates()
    df_directories['dest_path'] = df_directories['plateID']+'/'+df_directories['well']+'/'+'field_'+df_directories['field']
    
    for dest in df_directories['dest_path'].tolist():
        full_path = os.path.join(dest_path, dest)
        if not os.path.exists(full_path):
            os.makedirs(full_path)
    return df_directories 
        
dest_path = '/home/exacloud/gscratch/HeiserLab/images/'

print("starting file copy for "+plate_name)
df_all = get_file_df(plate_name)
#poor_quality_wells = ("A5", "A6", "B5", "B6","C5", "C6","D5", "D6")
#poor_quality_wells = ("A1", "A2", "A3", "A4", "A5","A6",
#                      "B1", "B2", "B3", "B4", "B5", "B6", 
#                      "C1", "C2",       "C4", "C5", "C5",
#                      "D1", "D2", "D3", "D4", "D5", "D6")

poor_quality_wells = ()

df = df_all[~df_all.well.isin(poor_quality_wells)].reset_index()
df_directories = make_directories(df, dest_path)
    
df = df.join(df_directories.set_index(['plateID','well','field']),on = ['plateID','well','field'])
print("copying files to "+dest_path+'/'+plate_name)
for count, src_path in enumerate(df['path'].tolist()):
    full_dest_path = os.path.join(dest_path, df.at[count,'dest_path'])
    shutil.copy2(src_path, full_dest_path)
print("done copying files for "+plate_name+"\n")

starting file copy for HC01401
getting files for HC01401
making directories in /home/exacloud/gscratch/HeiserLab/images/
copying files to /home/exacloud/gscratch/HeiserLab/images//HC01401
done copying files for HC01401



In [15]:
os.path.join(dest_path, df.at[count,'dest_path'])

'/home/exacloud/gscratch/HeiserLab/images/HC01301/C5/field_1'

In [13]:
count = 0